In [1]:
import os
from pathlib import Path
import numpy as np
import pywt
import matplotlib.pyplot as plt
import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

Using TensorFlow backend.


In [4]:
def readData(root_directory: str):
    p = Path(root_directory)
    x_path = list(p.glob('X*.txt'))[0]
    y_path = list(p.glob('y*.txt'))[0]
    x = np.loadtxt(x_path)
    y = np.loadtxt(y_path)
    return x,y

In [8]:
x, y = readData('data/train')
x_test, y_test = readData('data/test')

In [9]:
print(x.shape)
print(y.shape)
print(x_test.shape)
print(y_test.shape)

(7352, 561)
(7352,)
(2947, 561)
(2947,)


In [7]:
def myCWT(vector):
    wavelet = "morl"
    coeff, freqs = pywt.cwt(vector, np.arange(1,129), wavelet)
    # Im't not sure about the square power here
    cwtPower = coeff #abs(coeff)**2
#     fig, ax=plt.subplots(figsize=(12, 12))
#     im=ax.imshow(cwtPower[::-1],aspect='auto')
    return cwtPower

In [11]:
x_train = np.empty((len(x), 128, 561, 1))
for i, v in enumerate(x):
    if i % 1000 == 0:
        print(i)
    xi = myCWT(v)
    xi = xi.reshape(xi.shape[0], xi.shape[1], 1)
    x_train[i] = xi

0
1000
2000
3000
4000
5000
6000
7000


In [12]:
x_test1 = np.empty((len(x_test), 128, 561, 1))
for i, v in enumerate(x_test):
    if i % 100 == 0:
        print(i)
    xi = myCWT(v)
    xi = xi.reshape(xi.shape[0], xi.shape[1], 1)
    x_test1[i] = xi

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900


In [13]:
y_train = to_categorical([i-1 for i in y])
y_test1 = to_categorical([i-1 for i in y_test])

In [17]:
print(y_train[0])
print(y_test1[0])

[0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0.]


In [15]:
print(x_train.shape)
print(x_test1.shape)

(7352, 128, 561, 1)
(2947, 128, 561, 1)


In [19]:
import pickle
with open("x_train", "wb") as f:
    pickle.dump(x_train, f)
with open("y_train", "wb") as f:
    pickle.dump(y_train, f)
with open("x_test", "wb") as f:
    pickle.dump(x_test1, f)
with open("y_test", "wb") as f:
    pickle.dump(y_test1, f)

In [12]:
#create model
model = Sequential()#add model layers
model.add(Conv2D(64, kernel_size=(5, 5), 
                 activation='relu', 
                 input_shape=(128,561,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=1)

Epoch 1/3
7352/7352 [==============================] - 44s 6ms/step - loss: 0.8035 - accuracy: 0.7225
Epoch 2/3
7352/7352 [==============================] - 26s 3ms/step - loss: 0.2868 - accuracy: 0.8958
Epoch 3/3
7352/7352 [==============================] - 25s 3ms/step - loss: 0.1639 - accuracy: 0.9411
